# Tokenize and save data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.chdir('../../../')

In [3]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *

In [4]:
# from fastai.text import *

## Preparing the data

In [5]:
version = 'v20'
data_path = Path('data/midi')
version_path = data_path/version

In [6]:
source_dir = 'piano_duet'
# source_dir = 'midi_encode'
out_path = version_path/source_dir
csv_path = out_path/f'{source_dir}.csv'
version_path.ls()

[PosixPath('data/midi/v20/metadata'),
 PosixPath('data/midi/v20/midi_sources'),
 PosixPath('data/midi/v20/piano_duet'),
 PosixPath('data/midi/v20/s2s_encode')]

In [7]:
csv = pd.read_csv(csv_path)

/home/ubuntu/anaconda3/envs/musicautobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,3,4,7,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
csv = csv.loc[csv['numpy'].notna()];

In [9]:
def create_databunch(files, data_save_name, path=out_path):
    save_file = path/data_save_name
    if save_file.exists():
        data = load_data(path, data_save_name)
    else:
        save_file.parent.mkdir(exist_ok=True, parents=True)
        vocab = MusicVocab.create()
        processors = [OpenNPFileProcessor(), MusicItemProcessor()]

        data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
        data.save(data_save_name)
    return data

In [10]:
def get_files(csv):
    files = csv['numpy']
    flist = [Path(version_path/f) for f in files.values if isinstance(f, str)]
    flist = [f for f in flist if f.exists()]
    return flist

In [11]:
version_path

PosixPath('data/midi/v20')

### Create All Dataset

In [12]:
csv.head()

,parts,ht_time_signature,midi,section,midi_title,title,numpy,mxl,ht_bpm,md5,source,artist,ht_key,ht_mode,genres,song_url
0,"intro,chorus",4.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,chorus,yu-gi-oh3,yu-gi-oh-theme-song,piano_duet/hooktheory/pianoroll/w/wayne-sharpe...,NaN,128.0,bf1f29e5ff84e3e93e37fb873bfb590e,hooktheory,wayne-sharpe,C,1.0,NaN,https://www.hooktheory.com/theorytab/view/wayn...
1,"intro,chorus",3.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,intro,yu-gi-oh,yu-gi-oh-theme-song,piano_duet/hooktheory/pianoroll/w/wayne-sharpe...,NaN,85.0,055f80ad67f64edb14a85ca8fbfe8c29,hooktheory,wayne-sharpe,C,1.0,NaN,https://www.hooktheory.com/theorytab/view/wayn...
2,chorus,4.0,midi_sources/hooktheory/pianoroll/w/what-a-day...,chorus,kiefer,kiefer,piano_duet/hooktheory/pianoroll/w/what-a-day/k...,NaN,96.0,197f96f5d181f6ce1e2c5ab04ac1ff87,hooktheory,what-a-day,D,6.0,Jazz,https://www.hooktheory.com/theorytab/view/what...
3,"verse,pre-chorus,chorus",4.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,pre-chorus,senbonzakura - pre-Pre-Chorus,senbonzakura,piano_duet/hooktheory/pianoroll/w/whiteflame/s...,NaN,152.0,9e7ce13a35f1314423a9a6d5a5287a4a,hooktheory,whiteflame,D,6.0,"J-Pop,Pop",https://www.hooktheory.com/theorytab/view/whit...
4,"verse,pre-chorus,chorus",4.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,verse,Senbonzakura,senbonzakura,piano_duet/hooktheory/pianoroll/w/whiteflame/s...,NaN,152.0,d5aaf79d0989222f1362f9f46c540a27,hooktheory,whiteflame,D,6.0,"J-Pop,Pop",https://www.hooktheory.com/theorytab/view/whit...


In [13]:
all_files = get_files(csv); len(all_files)

113111

## Testing

In [14]:
import random
# sample_data = create_databunch(random.sample(all_files, 1000), data_save_name='cached/sample.pkl')

In [15]:
files = random.sample(all_files, 1000)
path = out_path

vocab = MusicVocab.create()
processors = [OpenNPFileProcessor(), MusicItemProcessor()]

data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)

item = data.train_dl.dataset.x[0]

type(item)

item.data

a = np.load(files[0])

a

files[0]

PosixPath('data/midi/v20/piano_duet/130k_reddit/W/W/withlove.npy')

In [16]:
np.load('data/midi/v20/piano_duet/130k_reddit/M/M/Maxime.npy')

array([[71,  4],
       [71,  7],
       [67,  4],
       [67,  7],
       ...,
       [69,  1],
       [64,  4],
       [61,  4],
       [57,  1]])

In [17]:
item.position

array([  0,   0,   0,   0, ..., 586, 586, 587, 587])

In [18]:
item.position

array([  0,   0,   0,   0, ..., 586, 586, 587, 587])

## End Testing

In [14]:
all_data = create_databunch(all_files, data_save_name='cached/all.pkl')

In [15]:
all_data.one_batch()

(tensor([[[ 61,  32],
          [145,  32],
          [  8,  32],
          ...,
          [153, 112],
          [ 63, 112],
          [153, 112]],
 
         [[ 61,  32],
          [145,  32],
          [  8,  32],
          ...,
          [153, 112],
          [ 63, 112],
          [153, 112]],
 
         [[ 61,  32],
          [145,  32],
          [  8,  32],
          ...,
          [153, 112],
          [ 63, 112],
          [153, 112]],
 
         ...,
 
         [[ 61,  32],
          [145,  32],
          [  8,  32],
          ...,
          [153, 112],
          [ 63, 112],
          [153, 112]],
 
         [[ 61,  32],
          [145,  32],
          [  8,  32],
          ...,
          [153, 112],
          [ 63, 112],
          [153, 112]],
 
         [[ 61,  32],
          [145,  32],
          [  8,  32],
          ...,
          [153, 112],
          [ 63, 112],
          [153, 112]]]), tensor([[[145,  32],
          [  8,  32],
          [145,  32],
          ...,
    

### Create sample

In [16]:
import random
sample_data = create_databunch(random.sample(all_files, 1000), data_save_name='cached/sample.pkl')

Note: we are reusing all_vocab for the following datasets

### Create Hooktheory Dataset

In [17]:
hook_csv = csv.loc[csv.source.isin(['hooktheory'])]
hook_files = get_files(hook_csv); len(hook_files)

19874

In [18]:
hook_data = create_databunch(hook_files, 'cached/hook.pkl')

### Create Hooktheory C Dataset

In [19]:
hook_csv = csv.loc[csv.source.isin(['hooktheory_c'])]
hook_files = get_files(hook_csv); len(hook_files)

20028

In [20]:
hook_data = create_databunch(hook_files, 'cached/hook_c.pkl')

## Low quality

In [21]:
lq_csv = csv.loc[csv.source.isin(['reddit', 'classical_piano', 'ecomp', 'midiworld', 'freemidi', 'lmd', 'cprato', 'wikifonia', 'classical_archives'])]
lq_files = get_files(lq_csv); len(lq_files)

64014

In [22]:
lq_data = create_databunch(lq_files, 'cached/lq.pkl')

## All High quality

In [23]:
hq_csv = csv.loc[csv.source.isin(['hooktheory', 'musescore'])]
hq_files = get_files(hq_csv); len(hq_files)

29069

In [24]:
hq_data = create_databunch(hq_files, 'cached/hq.pkl')

### Testing

In [30]:
data = all_data

In [38]:
data.train_ds.x

MusicItemList (100953 items)
xxbos xxpad xxsep d1 n55 d1 xxsep d1 n54 d1 xxsep d1 n45 d1 xxsep d1 n53 d1 xxsep d1 n52 d1 xxsep d1 n45 d1 xxsep d1 n53 d1 xxsep d1 n52 d1 xxsep d1 n45 d3 xxsep d3 n55 d1 xxsep d1 n54 d1 xxsep d1 n55 d1 xxsep d1 n61 d1 xxsep d1 n59 d16 n55 d16 n52 d16 xxsep d16 n60 d8 n57 d8 n55 d8 n52 d8 xxsep d8 n59 d8 n55 d8 n52 d8 xxsep d8 n50 d24 n46 d24 n43 d24 xxsep d24 n52 d4 n48 d4 n45 d4 n43 d4 xxsep d4 n54 d4 n50 d4 n47 d4 n45 d4 xxsep d4 n55 d16 n51 d16 n48 d16 xxsep d16 n62 d8 n60 d8 n56 d8 n53 d8 xxsep d8 n55 d8 n51 d8 n48 d8 xxsep d8 n58 d16 n54 d16 n51 d16,xxbos xxpad n65 d1 xxsep d1 n67 d14 xxsep d14 n67 d1 xxsep d1 n65 d1 xxsep d1 n67 d1 xxsep d1 n65 d6 xxsep d6 n64 d24 xxsep d28 n60 d4 xxsep d4 n67 d1 xxsep d1 n65 d1 xxsep d1 n67 d1 xxsep d1 n68 d9 xxsep d9 n67 d1 xxsep d1 n68 d1 xxsep d1 n67 d24 xxsep d24 n67 d2 xxsep d2 n70 d10 xxsep d10 n69 d1 xxsep d1 n70 d1 xxsep d1 n69 d1 xxsep d1 n67 d5 xxsep d5 n68 d1 xxsep d1 n67 d1 xxsep d1 n65 d8 xxsep d8 n67 

In [ ]:
# train_ids_file = out_path/'tmp/all/train_ids.npy'
# all_ids = np.load(train_ids_file)
# id_cat = np.concatenate(all_ids); id_cat.shape
# ax = tuple(range(len(id_cat.shape)-1))
# max_vocab = id_cat.max(axis=ax)
# max_vocab = (max_vocab+1).tolist(); max_vocab

In [ ]:
npenc2stream(data.train_ds.x[10]).show()